# Preprocess HyperNeRF data to NSFF data style

In [1]:
import json
import numpy as np
import imageio
import pdb
from pathlib import Path
import cv2
import os
import shutil
from PIL import Image

# @markdown The base directory for all captures. This can be anything if you're running this notebook on your own Jupyter runtime.
save_dir = './data/'  
capture_name = 'kubric_single_car_rand_v2' 
image_scale = '2x'
# The root directory for this capture.
root_dir = Path(save_dir, capture_name)
# Where to save RGB images.
rgb_dir = root_dir / 'rgb'
rgb_raw_dir = root_dir / 'rgb-raw'
# Where to save mask images.
mask_dir = root_dir / 'mask'
mask_raw_dir = root_dir / 'mask-raw'
camera_dir = root_dir / 'camera-gt'
if not camera_dir.exists():
    camera_dir = root_dir / 'camera'
camera_test_dir = root_dir / 'camera-paths-gt' / 'orbit-mild'
if not camera_test_dir.exists():
    camera_test_dir = root_dir / 'camera-paths' / 'orbit-mild'

# make dirs for freeze test if available
freeze_test_dir = root_dir / 'freeze-test'
if freeze_test_dir.exists():
    rgb_raw_freeze_test_dir = freeze_test_dir / 'rgb-raw'
    rgb_freeze_test_dir = freeze_test_dir / 'rgb'
    cam_freeze_test_dir = freeze_test_dir / 'camera-gt'
    if not cam_freeze_test_dir.exists():
        cam_freeze_test_dir = freeze_test_dir / 'camera'

    # with open(freeze_test_dir / 'metadata_kubric.json','r') as f:
    #         k_freeze_meta = json.load(f)

# with open(root_dir / 'metadata_kubric.json','r') as f:
#     k_meta = json.load(f)

tmp_rgb_raw_dir = rgb_raw_dir

rgb_list = list(sorted(rgb_raw_dir.glob( '*.png')))
image_size = imageio.imread(rgb_list[0]).shape[:2]
rgb_list[:3]

target_image_dir = root_dir / 'images'
target_image_dir.mkdir(exist_ok=True)
for rgb in rgb_list:
    shutil.copy(str(rgb), str(target_image_dir))


In [3]:
# read scene info
if (root_dir / 'scene_gt.json').exists():
    with open(root_dir / 'scene_gt.json','r') as f:
        scene_info = json.load(f)
else:
    with open(root_dir / 'scene.json','r') as f:
        scene_info = json.load(f)

near = scene_info['near'] / scene_info['scale']
far = scene_info['far'] / scene_info['scale']

# read camera info

c2w_mats = []
hwfs = []
bounds_mats = []
for i, rgb in enumerate(rgb_list):
    with open(camera_dir / f'{rgb.stem}.json','r') as f:
        camera_info = json.load(f)
    hwf = np.concatenate([camera_info['image_size'], [camera_info['focal_length']]])
    c2w_rot = np.array(camera_info['orientation']).T
    c2w_trans = camera_info['position']

    c2w_mat = np.zeros([4,4])
    c2w_mat[:3,:3] = c2w_rot
    c2w_mat[3,:3] = c2w_trans
    c2w_mat[3,3] = 1.
    c2w_mats.append(c2w_mat)
    hwfs.append(hwf[..., None])
    bounds_mats.append(np.array([near, far]))

c2w_mats = np.array(c2w_mats)
hwfs = np.array(hwfs).transpose([1,2,0])

poses = c2w_mats[:, :3, :4].transpose([1,2,0])
poses = np.concatenate([poses, hwfs], 1)

# must switch to [-y, x, z] from [x, -y, -z], NOT [r, u, -t]
poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], 
                        -poses[:, 2:3, :], 
                        poses[:, 3:4, :], 
                        poses[:, 4:5, :]], 1)

save_arr = []
for i in range((poses.shape[2])):
    save_arr.append(np.concatenate([poses[..., i].ravel(), bounds_mats[i]], 0))

save_arr = np.array(save_arr)
np.save(root_dir / 'poses_bounds.npy', save_arr)

with open( root_dir / 'scene.json', 'w') as f:
    json.dump(scene_info, f, indent=2)

In [2]:
# prepare for freeze test
freeze_rgb_list = sorted((freeze_test_dir / 'rgb' / image_scale).glob( '*.png'))
if freeze_test_dir.exists():
    target_image_dir = freeze_test_dir / 'images'
    target_image_dir.mkdir(exist_ok=True)
    for rgb in list(freeze_rgb_list):
        shutil.copy(str(rgb), str(target_image_dir))


    c2w_mats = []
    hwfs = []
    bounds_mats = []
    for i, rgb in enumerate(freeze_rgb_list):
        with open(cam_freeze_test_dir / f'{rgb.stem}.json','r') as f:
            camera_info = json.load(f)
        hwf = np.concatenate([camera_info['image_size'], [camera_info['focal_length']]])
        c2w_rot = np.array(camera_info['orientation']).T
        c2w_trans = camera_info['position']

        c2w_mat = np.zeros([4,4])
        c2w_mat[:3,:3] = c2w_rot
        c2w_mat[3,:3] = c2w_trans
        c2w_mat[3,3] = 1.
        c2w_mats.append(c2w_mat)
        hwfs.append(hwf[..., None])
        bounds_mats.append(np.array([near, far]))

    c2w_mats = np.array(c2w_mats)
    hwfs = np.array(hwfs).transpose([1,2,0])

    poses = c2w_mats[:, :3, :4].transpose([1,2,0])
    poses = np.concatenate([poses, hwfs], 1)

    # must switch to [-y, x, z] from [x, -y, -z], NOT [r, u, -t]
    poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], 
                            -poses[:, 2:3, :], 
                            poses[:, 3:4, :], 
                            poses[:, 4:5, :]], 1)

    save_arr = []
    for i in range((poses.shape[2])):
        save_arr.append(np.concatenate([poses[..., i].ravel(), bounds_mats[i]], 0))

    save_arr = np.array(save_arr)
    np.save(freeze_test_dir / 'poses_bounds.npy', save_arr)

